<a href="https://colab.research.google.com/github/abakm/AL-ML_Assignment-1/blob/master/CaseStudy03ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load the libaries

In [125]:
import os
import cv2
import kagglehub
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models

### Dowload dataset

In [126]:

path = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fruit-and-vegetable-image-recognition


# Set Target size for images and group items into categories

In [127]:
#categories
TARGET_SIZE = 64
fruits = ["banana", "apple", "pear", "grapes", "orange", "kiwi", "watermelon", "pomegranate", "pineapple", "mango"]
vegetables = ["cucumber", "carrot", "capsicum", "onion", "potato", "lemon", "tomato", "radish", "beetroot", "cabbage", "lettuce", "spinach", "soybean", "cauliflower", "bell pepper", "chilly", "pepper", "turnip", "corn", "sweetcorn", "sweet potato", "paprika", "jalapeño", "ginger", "garlic", "peas", "eggplant"]

## Load dataset from the kagglehub

In [128]:

train_images = list()
train_labels = list()

test_images = list()
test_labels = list()

for root, _, image_files in os.walk(path):
    for image_file in image_files:
      root_lower = root.lower()
      # Load only train and test samples
      if "train" in root_lower or "test" in root_lower:
        print("file:", os.path.join(root, image_file))

        img = cv2.imread(os.path.join(root, image_file))
        if img is not None:

          # resize the image
          img = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))

          # find whether its fruit or vegetable
          category = str(root.split("/")[-1]).lower()
          if category in fruits:
              label = 0
          elif category in vegetables:
              label = 1
          else:
              continue

          # Decide to whether the image needs to append(train/test)
          if "train" in root_lower:
              train_images.append(img)
              train_labels.append(label)
          else:
              test_images.append(img)
              test_labels.append(label)


file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_4.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_2.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_7.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_5.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_10.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_1.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_8.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_3.JPG
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_6.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/capsicum/Image_9.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/sweetcorn/Image_4.jpg
file: /kaggle/input/fruit-and-vegetable-image-recognition/test/sweetcorn/I

## Convert the train and test data into numpy array

In [129]:

train_images = np.array(train_images)
train_labels = np.array(train_labels)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

Scale data with respect to 255

In [130]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

### Create Convolution Network

In [131]:
model = models.Sequential()
# First hidden layer
model.add(layers.Conv2D(TARGET_SIZE, (3, 3), activation='relu', input_shape=(TARGET_SIZE, TARGET_SIZE, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Second hidden layer
model.add(layers.Conv2D(2*TARGET_SIZE, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Third hidden layer
model.add(layers.Conv2D(2*TARGET_SIZE, (3, 3), activation='relu'))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Create Fully Connected ANN

In [132]:
# Flatten and add dense layers
model.add(layers.Flatten())
model.add(layers.Dense(2*TARGET_SIZE, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

# Compile the model

In [133]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train the model

In [134]:
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


85/85 ━━━━━━━━━━━━━━━━━━━━ 77s 550ms/step - accuracy: 0.6218 - loss: 0.6829 - val_accuracy: 0.6796 - val_loss: 0.5941
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 80s 529ms/step - accuracy: 0.6918 - loss: 0.5850 - val_accuracy: 0.7120 - val_loss: 0.5377
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 84s 553ms/step - accuracy: 0.7139 - loss: 0.5504 - val_accuracy: 0.7573 - val_loss: 0.5437
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 85s 586ms/step - accuracy: 0.7226 - loss: 0.5421 - val_accuracy: 0.6828 - val_loss: 0.6757
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 78s 548ms/step - accuracy: 0.7337 - loss: 0.5440 - val_accuracy: 0.7670 - val_loss: 0.4847
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 87s 603ms/step - accuracy: 0.7370 - loss: 0.5092 - val_accuracy: 0.8058 - val_loss: 0.4643
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 76s 532ms/step - accuracy: 0.7740 - loss: 0.4579 - val_accuracy: 0.7961 - val_loss: 0.4311
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 45s 530ms/step - accuracy: 0.7642 - loss: 0.4775 - val_accuracy: 0.841

# Evaluate model

In [135]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test accuracy: {test_acc}')

10/10 - 1s - 123ms/step - accuracy: 0.8770 - loss: 0.3425
Test accuracy: 0.8770226240158081


## Predict the model

In [136]:
print("test_label: ", test_labels[23])
prediction =model.predict(test_images[23].reshape(1, TARGET_SIZE, TARGET_SIZE, 3))
print("Prediction: ", prediction)
predicted_class = np.argmax(prediction)
print("Vegetable/fruit:", "Vegetable" if predicted_class else "Fruit")

test_label:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Prediction:  [[0.5837711 0.4162289]]
Vegetable/fruit: Fruit
